<a href="https://colab.research.google.com/github/xiaochengJF/MachineLearning/blob/master/%E5%86%B3%E7%AD%96%E6%A0%91_CART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# 对y的各种可能的取值出现的个数进行计数。其他函数利用该函数来计算数据集和的混杂程度
def uniquecounts(rows):
    results = {}
    for row in rows:
        #计数结果在最后一列
        r = row[len(row)-1]
        if r not in results:results[r] = 0
        results[r]+=1
    return results # 返回一个字典


# 熵
def entropy(rows):
    from math import log
    log2 = lambda x:log(x)/log(2)
    results = uniquecounts(rows)
    #开始计算熵的值
    ent = 0.0
    for r in results.keys():
        p = float(results[r])/len(rows)
        ent = ent - p*log2(p)
    return ent

In [0]:
#定义节点的属性
class decisionnode:
    def __init__(self,col = -1,value = None, results = None, tb = None,fb = None):
        self.col = col   # col是待检验的判断条件所对应的列索引值
        self.value = value # value对应于为了使结果为True，当前列必须匹配的值
        self.results = results #保存的是针对当前分支的结果，它是一个字典
        self.tb = tb ## desision node,对应于结果为true时，树上相对于当前节点的子树上的节点
        self.fb = fb ## desision node,对应于结果为true时，树上相对于当前节点的子树上的节点



# 基尼不纯度


# 随机放置的数据项出现于错误分类中的概率
def giniimpurity(rows):
    total = len(rows)
    counts = uniquecounts(rows)
    imp =0
    for k1 in counts:
        p1 = float(counts[k1])/total
        for k2 in counts: # 这个循环是否可以用（1-p1）替换？
            if k1 == k2: continue
            p2 = float(counts[k2])/total
            imp+=p1*p2
    return imp


# 改进giniimpurity


def giniimpurity_2(rows):
    total = len(rows)
    counts = uniquecounts(rows)
    imp = 0
    for k1 in counts.keys():
        p1 = float(counts[k1])/total
        imp+= p1*(1-p1)
    return imp



#在某一列上对数据集进行拆分。可应用于数值型或因子型变量
def divideset(rows, column, value):
    #定义一个函数，判断当前数据行属于第一组还是第二组
    split_function = None
    if isinstance(value,int) or isinstance(value,float):
        split_function = lambda row:row[column] >= value
    else:
        split_function = lambda row:row[column] == value
    # 将数据集拆分成两个集合，并返回
    set1 = [row for row in rows if split_function(row)]
    set2 = [row for row in rows if not split_function(row)]
    return(set1,set2)


# 以递归方式构造树

def buildtree(rows,scoref = entropy):
    if len(rows)==0 : return decisionnode()
    current_score = scoref(rows)
    
    # 定义一些变量以记录最佳拆分条件
    best_gain = 0.0
    best_criteria = None
    best_sets = None
    
    column_count = len(rows[0]) - 1
    for col in range(0,column_count):
        #在当前列中生成一个由不同值构成的序列
        column_values = {}
        for row in rows:
            column_values[row[col]] = 1 # 初始化
        #根据这一列中的每个值，尝试对数据集进行拆分
        for value in column_values.keys():
            (set1,set2) = divideset(rows,col,value)
            
            # 信息增益
            p = float(len(set1))/len(rows)
            gain = current_score - p*scoref(set1) - (1-p)*scoref(set2)
            if gain>best_gain and len(set1)>0 and len(set2)>0:
                best_gain = gain
                best_criteria = (col,value)
                best_sets = (set1,set2)
                
    #创建子分支
    if best_gain>0:
        trueBranch = buildtree(best_sets[0])  #递归调用
        falseBranch = buildtree(best_sets[1])
        return decisionnode(col = best_criteria[0],value = best_criteria[1],
                            tb = trueBranch,fb = falseBranch)
    else:
        return decisionnode(results = uniquecounts(rows))

# 决策树的显示
def printtree(tree,indent = ''):
    # 是否是叶节点
    if tree.results != None:
        print str(tree.results)
    else:
        # 打印判断条件
        print str(tree.col)+":"+str(tree.value)+"? "
        #打印分支
        print indent+"T->",
        printtree(tree.tb,indent+" ")
        print indent+"F->",
        printtree(tree.fb,indent+" ")


# 对新的观测数据进行分类


def classify(observation,tree):
    if tree.results!= None:
        return tree.results
    else:
        v = observation[tree.col]
        branch = None
        if isinstance(v,int) or isinstance(v,float):
            if v>= tree.value: branch = tree.tb
            else: branch = tree.fb
        else:
            if v==tree.value : branch = tree.tb
            else: branch = tree.fb
        return classify(observation,branch)

In [0]:
# test
my_data=[['slashdot','USA','yes',18,'None'],
        ['google','France','yes',23,'Premium'],
        ['digg','USA','yes',24,'Basic'],
        ['kiwitobes','France','yes',23,'Basic'],
        ['google','UK','no',21,'Premium'],
        ['(direct)','New Zealand','no',12,'None'],
        ['(direct)','UK','no',21,'Basic'],
        ['google','USA','no',24,'Premium'],
        ['slashdot','France','yes',19,'None'],
        ['digg','USA','no',18,'None'],
        ['google','UK','no',18,'None'],
        ['kiwitobes','UK','no',19,'None'],
        ['digg','New Zealand','yes',12,'Basic'],
        ['slashdot','UK','no',21,'None'],
        ['google','UK','yes',18,'Basic'],
        ['kiwitobes','France','yes',19,'Basic']]



divideset(my_data,2,'yes')

([['slashdot', 'USA', 'yes', 18, 'None'],
  ['google', 'France', 'yes', 23, 'Premium'],
  ['digg', 'USA', 'yes', 24, 'Basic'],
  ['kiwitobes', 'France', 'yes', 23, 'Basic'],
  ['slashdot', 'France', 'yes', 19, 'None'],
  ['digg', 'New Zealand', 'yes', 12, 'Basic'],
  ['google', 'UK', 'yes', 18, 'Basic'],
  ['kiwitobes', 'France', 'yes', 19, 'Basic']],
 [['google', 'UK', 'no', 21, 'Premium'],
  ['(direct)', 'New Zealand', 'no', 12, 'None'],
  ['(direct)', 'UK', 'no', 21, 'Basic'],
  ['google', 'USA', 'no', 24, 'Premium'],
  ['digg', 'USA', 'no', 18, 'None'],
  ['google', 'UK', 'no', 18, 'None'],
  ['kiwitobes', 'UK', 'no', 19, 'None'],
  ['slashdot', 'UK', 'no', 21, 'None']])

In [0]:
giniimpurity(my_data)

0.6328125

In [0]:
giniimpurity_2(my_data)

0.6328125

In [0]:
tree = buildtree(my_data)

In [0]:
printtree(tree = tree)

0:google? 
T-> 3:21? 
 T-> {'Premium': 3}
 F-> 2:yes? 
  T-> {'Basic': 1}
  F-> {'None': 1}
F-> 0:slashdot? 
 T-> {'None': 3}
 F-> 2:yes? 
  T-> {'Basic': 4}
  F-> 3:21? 
   T-> {'Basic': 1}
   F-> {'None': 3}


In [0]:
classify(['(direct)','USA','yes',5],tree)

{'Basic': 4}

In [0]:
# 决策树的剪枝
def prune(tree,mingain):
    # 如果分支不是叶节点，则对其进行剪枝
    if tree.tb.results == None:
        prune(tree.tb,mingain)
    if tree.fb.results == None:
        prune(tree.fb,mingain)
    # 如果两个子分支都是叶节点，判断是否能够合并
    if tree.tb.results !=None and tree.fb.results !=None:
        #构造合并后的数据集
        tb,fb = [],[]
        for v,c in tree.tb.results.items():
            tb+=[[v]]*c
        for v,c in tree.fb.results.items():
            fb+=[[v]]*c
        #检查熵的减少量
        delta = entropy(tb+fb)-(entropy(tb)+entropy(fb)/2)
        if delta < mingain:
            # 合并分支
            tree.tb,tree.fb = None,None
            tree.results = uniquecounts(tb+fb)
# test
tree = buildtree(my_data,scoref = giniimpurity)

In [0]:
prune(tree,0.1)
printtree(tree)

0:google? 
T-> 3:21? 
 T-> {'Premium': 3}
 F-> 2:yes? 
  T-> {'Basic': 1}
  F-> {'None': 1}
F-> 0:slashdot? 
 T-> {'None': 3}
 F-> 2:yes? 
  T-> {'Basic': 4}
  F-> 3:21? 
   T-> {'Basic': 1}
   F-> {'None': 3}


# CART_Classification

In [0]:
from numpy import *

In [0]:
def loadDataSet():
    dataSet = [[0, 0, 0],
               [0, 1, 1],
               [0, 2, 0],
               [1, 0, 1],
               [1, 1, 1],
               [1, 2, 1],
               [2, 0, 0],
               [2, 1, 1],
               [2, 2, 0]]
    labels = ['color','shape']
    return dataSet, labels

## 计算基尼指数
基尼指数（基尼不纯度）= 样本被选中的概率 * 样本被分错的概率
$$Gini(D) = \sum_{k=1}^{K}p_{k}(1-p_{k}) = 1 - \sum_{k=1}^{K}p_{k}^{2}=1 - \sum_{k=1}^{K}\frac{|D_{k}|}{|D|}$$

- $p_k$表示选中的样本属于k类别的概率，则这个样本被分错的概率是$1-p_k$

- 样本集合中有$K$个类别，一个随机选中的样本可以属于这$K$个类别中的任意一个，因而对类别求和

- 当为二分类时Gini(P) = 2p(1-p)

<font face=楷体 color=yellow size=5>疑问</font>  

In [0]:
def calcGini(dataSet):
    totalNum = shape(dataSet)[0]
    labelNum = {}
    gini = 0
    for data in dataSet:
        label = data[-1]
        if label in labelNum:
            labelNum[label] += 1
        else:
            labelNum[label] = 1

    for key in labelNum:
        p = labelNum[key] / totalNum
        gini += p * (1 - p)
    return gini

## 选择最优特征

In [0]:
def chooseBestFeatVal2Split(dataSet):
    #如果没有可划分的特征或所有目标变量相等，停止
    if(len(dataSet[0]) == 1): return None, None
    if(len(set([d[-1] for d in dataSet])) == 1): return None, None
    bestFeature = 0
    bestValue = 0
    lowestGini = 100000
    totalGini = calcGini(dataSet)
    totalNum = shape(dataSet)[0]
    for feature in range(shape(dataSet)[1] - 1):
        allValues = [d[feature] for d in dataSet]
        values = set(allValues)
        for value in values:
            leftChild, rightChild = splitByFeatVal(feature, value, dataSet)
            if(shape(leftChild)[0] == 0 or shape(rightChild)[0] == 0): continue
            leftNum = shape(leftChild)[0]
            rightNum = shape(rightChild)[0]
            curGini = leftNum / totalNum * calcGini(leftChild) + \
                      rightNum / totalNum * calcGini(rightChild)
            if(curGini < lowestGini):
                bestFeature = feature
                bestValue = value
                lowestGini = curGini
    #如果gini减少很小，停止
    if(totalGini - lowestGini < 0.00001): return None, None
    return bestFeature, bestValue

## 按特征划分数据集
如果 feature 对应的属性值等于 value 值，则将该条数据划分到左子树；如果 feature 对应的属性值不等于 value 值，则将该条数据划分到右子树

In [0]:
def splitByFeatVal(feature, value, dataSet):
    #左子树的值大于根节点的值
    dataSet = mat(dataSet)
    leftChild = dataSet[nonzero(dataSet[:,feature] == value)[0], :].tolist()
    #右子树的值小于等于根节点的值
    rightChild = dataSet[nonzero(dataSet[:,feature] != value)[0], :].tolist()

    return leftChild, rightChild

## 结束条件

checkIsOneCateg 函数用来判断数据集的目标变量是否为一个分类结果  
majorityCateg 函数用来选出目标变量中的大多数值作为输出变量

In [0]:
def checkIsOneCateg(newDataSet):
    flag = False
    categoryList = [data[-1] for data in newDataSet]
    category = set(categoryList)
    if(len(category) == 1):
        flag = True
    return flag

def majorityCateg(newDataSet):
    categCount = {}
    categList = [data[-1] for data in newDataSet]
    for c in categList:
        if c not in categCount:
            categCount[c] = 1
        else:
            categCount[c] += 1
    sortedCateg = sorted(categCount.items(), key = lambda x:x[1], reverse = True)

    return sortedCateg[0][0]

## 创建分类树

In [0]:
def createClassifTree(dataSet):
    feature, value = chooseBestFeatVal2Split(dataSet)
    #如果无法分割，那么返回叶节点的值,即所有目标变量相同则为此值，不同则为多数值
    if feature == None and checkIsOneCateg(dataSet):
        return dataSet[0][-1]
    if feature == None and not checkIsOneCateg(dataSet):
        return majorityCateg(dataSet)
    #如果可以继续分割，那么继续创建新的子树
    classifTree = {}
    classifTree['featIndex'] = feature
    classifTree['value'] = value
    leftChild, rightChild = splitByFeatVal(feature, value, dataSet)
    classifTree['leftChild'] = createClassifTree(leftChild)
    classifTree['rightChild'] = createClassifTree(rightChild)

    return classifTree

if __name__ == '__main__':
    dataSet, labels = loadDataSet()
    classifTree = createClassifTree(dataSet)
    print(classifTree)

# CART_Regression

In [0]:
from numpy import *

## 数据集
数据集共 25 条数据。第一列数据代表房子的评价得分，此数据集中所有数据的评价得分都是 5.23。第二列数据代表房子的平方数，第三列数据代表房子的价格，以万为单位  
5.23	1	0.1  
5.23	2	0.12  
5.23	3	0.02  
5.23	4	0.03  
5.23	5	0.12  
5.23	6	5.0  
5.23	7	5.2  
5.23	8	5.1  
5.23	9	5.02  
5.23	10	5.03  
5.23	11	10.8  
5.23	12	10.06  
5.23	13	10.03  
5.23	14	10.02  
5.23	15	10.44  
5.23	16	15.88  
5.23	17	15.06  
5.23	18	15.04  
5.23	19	15.30  
5.23	20	15.05  
5.23	21	20.8  
5.23	22	20.16  
5.23	23	20.24  
5.23	24	20.05  
5.23	25	20.09  

In [0]:
def loadDataSet():
    dataSet = []
    f = open('regData.txt')
    fr = f.readlines()
    for line in fr:
        line = line.strip().split('\t')
        linef = [float(li) for li in line]
        dataSet.append(linef)
    dataSetMat = mat(dataSet)
    return dataSetMat

## 计算平方误差

In [0]:
def calcErr(dataSetMat):
    error = var(dataSetMat[:,-1]) * shape(dataSetMat)[0]
    return error

## 选择最优特征

对每个特征的每个属性值，计算按该属性值二分后的两个子数据集的平方误差和，选择平方误差和最小的特征作为最优特征。除了用平方误差代替基尼指数之外，其他过程和分类树基本相同

In [0]:
def chooseBestFeatVal2Split(dataSetMat):
    #如果所有目标变量相等，停止
    if(len(set(dataSetMat[:,-1].T.tolist()[0])) == 1): return None, None
    bestFeature = 0
    bestValue = 0
    lowestErr = 100000
    totalErr = calcErr(dataSetMat)
    for feature in range(shape(dataSetMat)[1] - 1):
        allValues = [d[feature] for d in dataSetMat.tolist()]
        values = set(allValues)
        for value in values:
            leftChild, rightChild = splitByFeatVal(feature, value, dataSetMat)
            if(shape(leftChild)[0] == 0 or shape(rightChild)[0] == 0): continue
            curErr = calcErr(leftChild) + calcErr(rightChild)
            if(curErr < lowestErr):
                bestFeature = feature
                bestValue = value
                lowestErr = curErr
    #如果误差减少很小，停止
    if(totalErr - lowestErr < 1): return None, None
    return bestFeature, bestValue

 ## 按特征划分数据集

In [0]:
def splitByFeatVal(feature, value, dataSetMat):
    #左子树的值大于根节点的值
    leftChild = dataSetMat[nonzero(dataSetMat[:,feature] > value)[0], :]
    #右子树的值小于等于根节点的值
    rightChild = dataSetMat[nonzero(dataSetMat[:,feature] <= value)[0], :]

    return leftChild, rightChild

## 创建回归树

In [24]:
def createRegTree(dataSetMat):
    feature, value = chooseBestFeatVal2Split(dataSetMat)
    #如果无法分割，那么返回叶节点的值，即所有dataSetMat的均值
    if feature == None: return mean(dataSetMat[:,-1])
    #如果可以继续分割，那么继续创建新的子树
    regTree = {}
    regTree['featIndex'] = feature
    regTree['value'] = value
    leftChild, rightChild = splitByFeatVal(feature, value, dataSetMat)
    regTree['leftChild'] = createRegTree(leftChild)
    regTree['rightChild'] = createRegTree(rightChild)

    return regTree

if __name__ == '__main__':
    dataSetMat = loadDataSet()
    regTree = createRegTree(dataSetMat)
    print(regTree)

{'leftChild': {'leftChild': 20.268, 'featIndex': 1, 'value': 20.0, 'rightChild': {'leftChild': 15.266, 'featIndex': 1, 'value': 15.0, 'rightChild': 10.27}}, 'featIndex': 1, 'value': 10.0, 'rightChild': {'leftChild': 5.07, 'featIndex': 1, 'value': 5.0, 'rightChild': 0.078}}
